In [1]:
import sys
sys.path.append('D:\PyCharmProjects\VFPUMC02')
%load_ext autoreload
%autoreload 2

In [2]:
from datasets.DataSet import BankDataset
from consts.Constants import DATASETS_PATH
from enums.SplitRatio import SplitRatio
from enums.HideRatio import HideRatio
from configs.Config import VFPUConfig
from classfiers.Bagging_PU import Bagging_PU
from configs.Config import PUConfig
from classfiers.PU import PU

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, classification_report

from utils.DataProcessUtils import evaluate_model
from utils.DataProcessUtils import value_counts_for_labels

In [3]:
bankDataset = BankDataset()
bankDataset.get_hidden_labels(HideRatio.RATIO_0_2)
PUConfig['num_p_in_hidden'] = bankDataset.count_hidden_positives()
bankDataset.datasetInfo()

gbdt_classifier = GradientBoostingClassifier()

bagging_pu = Bagging_PU(base_estimator=gbdt_classifier,config=PUConfig)

bagging_pu.fit(bankDataset.df.values, bankDataset.hidden_y.values)

pu = PU(bagging_pu,PUConfig)
pu.fit(*bankDataset.get_train_X_y())
y_true = bankDataset.y.values
y_pred = pu.get_predicted_labels()
y_prob = pu.get_predict_proba()

result = evaluate_model(y_true, y_pred, y_prob)


2024-09-24 12:03:35,732 - INFO - 正在加载数据集 bank.csv
2024-09-24 12:03:35,932 - INFO - 数据集 bank.csv 加载完成，数据集形状为 (41188, 64)
2024-09-24 12:03:35,934 - INFO - Bagging_PU 初始化完成
2024-09-24 12:03:35,936 - INFO - 找到隐藏标签缓存文件 bank_hidden_RATIO_0_2.pkl，正在加载...
2024-09-24 12:03:35,938 - INFO - 隐藏标签已从缓存文件 bank_hidden_RATIO_0_2.pkl 加载
2024-09-24 12:03:35,942 - INFO - 正在获取数据集信息...
2024-09-24 12:03:35,944 - INFO - 标签y总数量为：41188, 隐藏标签的数量为：8237，比例为 0.2
2024-09-24 12:03:35,946 - INFO - 有标签的数量为：32951, 标签为1的数量为：3718, 标签为0的数量为：29233
2024-09-24 12:03:35,947 - INFO - 数据集的形状为: (41188, 64)
2024-09-24 12:03:35,955 - INFO - 开始训练 Bagging_PU 模型
2024-09-24 12:03:35,972 - INFO - 正样本数量: 3718, 负样本数量: 29233, 差值: -25515
2024-09-24 12:03:35,980 - INFO - 未标记样本数量: 8237, 样本差值: 25515
2024-09-24 12:03:35,982 - INFO - 未标记样本不足以平衡数据集，直接训练


{'n_estimators': 1,
 'n_iter': 500,
 'theta': 0.04,
 'theta_P': 0.04,
 'theta_N': 0.001,
 'num_p_in_hidden': 922}

TypeError: type object argument after * must be an iterable, not Bagging_PU

2024-09-24 12:04:34,323 - INFO - 开始训练 PU 模型
2024-09-24 12:04:34,330 - INFO - 开始初次训练 Bagging_PU 模型
2024-09-24 12:04:34,331 - INFO - 开始训练 Bagging_PU 模型
2024-09-24 12:04:34,351 - INFO - 正样本数量: 3718, 负样本数量: 29233, 差值: -25515
2024-09-24 12:04:34,359 - INFO - 未标记样本数量: 8237, 样本差值: 25515
2024-09-24 12:04:34,360 - INFO - 未标记样本不足以平衡数据集，直接训练
2024-09-24 12:04:36,894 - INFO - 获取out-of-bag得分
2024-09-24 12:04:36,895 - INFO - 初次训练完成，out-of-bag 预测得分已保存
2024-09-24 12:04:36,895 - INFO - 开始第 1 次迭代
2024-09-24 12:04:36,911 - INFO - 标签总数量: 41188，正样本标签数量: 3718, 负样本标签数量: 29233, 未标记样本数量: 8237
2024-09-24 12:04:36,917 - INFO - 根据阈值 0.04 找到 329 个新的正样本 (RP) 和 8 个新的负样本 (RN)
2024-09-24 12:04:36,918 - INFO - 第 1 次迭代更新了标签
2024-09-24 12:04:36,920 - INFO - 找到 329 个新的正样本
2024-09-24 12:04:36,922 - INFO - 找到 8 个新的负样本
2024-09-24 12:04:36,924 - INFO - 随机选择了 329 个样本
2024-09-24 12:04:36,925 - INFO - 保存了当前的预测结果
2024-09-24 12:04:36,926 - INFO - 更新了新找到的正样本列表，共 329 个正样本
2024-09-24 12:04:36,927 - INFO - 更新了新找到的负样本列表，共 8 个负样本
2024-09

In [16]:
y_prob

array([0.00173288, 0.00171991, 0.0016875 , ..., 0.00135869, 0.7994471 ,
       0.00091291])

In [15]:
y_pred

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [14]:
value_counts_for_labels(y_true, y_pred)

标签数组 1:
  值 0 出现了 36548 次
  值 1 出现了 4640 次

标签数组 2:
  值 0 出现了 36522 次
  值 1 出现了 4666 次



In [32]:
bankDataset.hidden_y[pu.iP_new]

40962    1
8196     1
30728    1
16392    1
40971    1
        ..
10227    1
26614    1
36856    1
40954    1
40955    1
Name: y, Length: 948, dtype: int64